In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Make sure that caffe is on the python path:
caffe_root = '../'  # this file is expected to be in {caffe_root}/examples
import sys
sys.path.insert(0, caffe_root + 'python')

import caffe

caffe.set_device(0)
caffe.set_mode_gpu()

caffe_root = '/home/cv/hdl/caffe'
training = '{}/training/pascalpart-fcn32s'.format(caffe_root)

In [2]:
# Load the original network and extract the fully connected layers' parameters.
net = caffe.Net('../models/vgg16/VGG_ILSVRC_16_layers_deploy.prototxt', 
                '../models/vgg16/VGG_ILSVRC_16_layers.caffemodel', 
                caffe.TEST)
params = ['fc6', 'fc7']
# params = [pr for pr in net.params.keys() if 'fc8' not in pr]
# fc_params = {name: (weights, biases)}
fc_params = {pr: (net.params[pr][0].data, net.params[pr][1].data) for pr in params}

for fc in params:
    print '{} weights are {} dimensional and biases are {} dimensional'.format(fc, fc_params[fc][0].shape, fc_params[fc][1].shape)


fc6 weights are (4096, 25088) dimensional and biases are (4096,) dimensional
fc7 weights are (4096, 4096) dimensional and biases are (4096,) dimensional


In [3]:
# Load the fully convolutional network to transplant the parameters.
net_full_conv = caffe.Net('{}/person/head/train.prototxt'.format(training),
                          caffe.TEST)
solver = caffe.SGDSolver('{}/person/head/model/solver.prototxt'.format(training))
params_full_conv = ['fc6', 'fc7']
# params_full_conv = [pr for pr in net_full_conv.params.keys() if 'score' not in pr]
# conv_params = {name: (weights, biases)}
conv_params = {pr: (net_full_conv.params[pr][0].data, net_full_conv.params[pr][1].data) for pr in params_full_conv}

for conv in params_full_conv:
    print '{} weights are {} dimensional and biases are {} dimensional'.format(conv, conv_params[conv][0].shape, conv_params[conv][1].shape)

fc6 weights are (4096, 512, 7, 7) dimensional and biases are (4096,) dimensional
fc7 weights are (4096, 4096, 1, 1) dimensional and biases are (4096,) dimensional


In [4]:
for pr, pr_conv in zip(params, params_full_conv):
    conv_params[pr_conv][0].flat = fc_params[pr][0].flat  # flat unrolls the arrays
    conv_params[pr_conv][1][...] = fc_params[pr][1]

Next, save the new model weights.

In [5]:
net_full_conv.save('../training/pascalpart-fcn32s/person/xavier_w_bilinear_vgg16fc.caffemodel')

In [6]:
print net_full_conv.params['conv1_1'][0].data

[[[[ 0.25916877 -0.13925445  0.24649265]
   [ 0.09279403  0.14464113 -0.30822614]
   [ 0.01478469  0.1358164   0.21603623]]

  [[-0.22692929 -0.06267941 -0.19831759]
   [ 0.09108618  0.18931279  0.19261226]
   [ 0.20259199  0.06351641  0.00712711]]

  [[ 0.18404546  0.11136398 -0.30799305]
   [ 0.00252679  0.31519625  0.08149803]
   [-0.04564217 -0.09791774 -0.13370386]]]


 [[[-0.14407367  0.01042938 -0.26521158]
   [ 0.00628915 -0.33142284  0.25094244]
   [ 0.13306841  0.09720412  0.02138585]]

  [[ 0.30945453  0.04634127 -0.05893001]
   [ 0.1907132   0.02065486  0.31245592]
   [ 0.32478908  0.0125587  -0.09336098]]

  [[-0.02415955 -0.28994787  0.29045674]
   [-0.26795888 -0.09223054 -0.26501697]
   [-0.2261124  -0.22427616 -0.05292854]]]


 [[[ 0.01076689 -0.14725414  0.00784254]
   [-0.14869089  0.12163162  0.1288147 ]
   [ 0.25504896 -0.27324128 -0.22855178]]

  [[-0.03750649  0.02560547  0.04597911]
   [ 0.20515385 -0.250819    0.17449966]
   [ 0.03175119 -0.20552893 -0.13631243